In [58]:
import pandas as pd

df = pd.read_csv("job_full_details.csv", encoding="ISO-8859-1", usecols=['ref_number','description'])
# df['description'] = df['description'].str.replace('\d+', '')
df.head(5)

,ref_number,description
0,758636,"â¬ global tote>Â» lankaneo we are 5\a i | ""hi..."
1,758633,now hiring!eee va no kvblwe are looking for so...
2,758632,ee eee ty j ra 7a â¢ if ad zo$ eae da0 us 7 e...
3,758612,rad.lk is a brand-new ecommerce platform. we s...
4,758587,taiinfor we are hiring!infor is a global compa...


In [59]:
import spacy
from spacy.matcher import Matcher

nlp = spacy.load("en_core_web_sm")

In [61]:
csharp_pattern1= [{'LOWER': 'c#'}]
csharp_pattern2= [{'LOWER': 'c'},{'LOWER': '#'}]
csharp_pattern3= [{'LOWER': 'c'},{'LOWER': 'sharp'}]

obj_c_pattern1 = [{'LOWER': 'objective'},{'IS_PUNCT': True, 'OP': '?'},{'LOWER': 'c'}]
obj_c_pattern2 = [{'LOWER': 'objectivec'}]

go_pattern1    = [{'LOWER': 'go', 'POS': {'NOT_IN': ['VERB']}}]
go_pattern2    = [{'LOWER': 'golang'}]

sql_pattern    = [{'LOWER': {'IN': ['sql', 'mysql']}}]
mongo_pattern  = [{'LOWER': 'mongo'},{'LOWER': 'mongodb'}]

asp_net_pattern = [{'LOWER':{'IN':['asp.net','asp']}}]
js_pattern     = [{'LOWER': {'IN': ['js', 'javascript']}}]

frontend_pattern = [{'LOWER': {'IN': ['react', 'react.js','angular', 'angular.js','vue', 'vue.js']}}]

languages = ['ruby', 'php', 'python', 'perl', 'java', '.net', 
                       'scala', 'c', 'cpp', 'html', 'css', 'kotlin', 'swift', 'android', 'ios']

def create_versioned(name):
    return [
        [{'LOWER': name}], 
        [{'LOWER': {'REGEX': f'({name}\d+\.?\d*.?\d*)'}}], 
        [{'LOWER': name}, {'TEXT': {'REGEX': '(\d+\.?\d*.?\d*)'}}],
    ]

flatten = lambda l: [item for sublist in l for item in sublist]
lang_patterns = flatten([create_versioned(lang) for lang in languages])

matcher = Matcher(nlp.vocab, validate=True)
matcher.add("PROG_LANG", None, 
            obj_c_pattern1, obj_c_pattern2,
            go_pattern1, go_pattern2,
            sql_pattern, mongo_pattern,
            csharp_pattern1, csharp_pattern2, csharp_pattern3,
            asp_net_pattern,
            frontend_pattern, js_pattern,
            *lang_patterns)

# matcher = Matcher(nlp.vocab, validate=True)

# matcher.add("OBJ_C_LANG", None, obj_c_pattern1, obj_c_pattern2)
# matcher.add("CSHARP_LANG", None, csharp_pattern1, csharp_pattern2, csharp_pattern3)
# matcher.add("PYTHON_LANG", None, python_pattern)
# matcher.add("JAVA_LANG", None, java_pattern)
# matcher.add("C_LANG", None, c_pattern)
# matcher.add("CPP_LANG", None, cpp_pattern)
# matcher.add("ASP_LANG", None, asp_net_pattern)
# matcher.add("ASP_LANG", None, asp_net_pattern)
# matcher.add("PHP_LANG", None, php_pattern)
# matcher.add("PERL_LANG", None, perl_pattern)
# matcher.add("RUBY_LANG", None, ruby_pattern)
# # matcher.add("GO_LANG", None, go_pattern1, go_pattern2)
# matcher.add("JS_LANG", None, js_pattern)
# matcher.add("HTML_LANG", None, html_pattern)
# matcher.add("CSS_LANG", None, css_pattern)
# matcher.add("SQL_LANG", None, sql_pattern)
# matcher.add("MONGO_LANG", None, mongo_pattern)

In [62]:
# Below is a custom code that will highlight the output of the matcher.
from IPython.display import HTML as html_print

def style(s, bold=False):
    blob = f"<text>{s}</text>"
    if bold:
        blob = f"<b style='background-color: #fff59d'>{blob}</b>"
    return blob

def html_generator(g, n=10):
    blob = ""
    for i in range(n):
        doc = next(g)

        state = [[t, False] for t in doc]
        for idx, start, end in matcher(doc):
            for i in range(start, end):
                state[i][1] = True
        blob += style(' '.join([style(str(t[0]), bold=t[1]) for t in state]) + '<br>') 
    return blob

In [63]:
descriptions = (_ for _ in df['description'].astype('unicode'))
# if a match is available in each description
g = (d for d in nlp.pipe(descriptions) if len(matcher(d)) > 0)
html_print(html_generator(g, n=10))

In [64]:
# total matches
titles = (_ for _ in df['description'].astype('unicode'))
sum(1 for d in nlp.pipe(titles) if len(matcher(d)) > 0)

195